In [3]:

import psycopg2

# 数据库配置
DB_NAME     = "fruit_chain"
DB_USER     = "postgres"
DB_PASSWORD = "lzq010921"
DB_HOST     = "localhost"
DB_PORT     = "5432"

def main():
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD,
        host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS batches (
      batch_id      BIGINT PRIMARY KEY,
      metadata      TEXT,
      current_owner TEXT,
      farmer        TEXT
    );
    """)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS stages (
      batch_id      BIGINT,
      stage_type    TEXT,
      location      TEXT,
      timestamp     BIGINT,
      actor_address TEXT,
      UNIQUE(batch_id, stage_type, timestamp)
    );
    """)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS ownership_transfers (
      batch_id     BIGINT,
      from_address TEXT,
      to_address   TEXT,
      UNIQUE(batch_id, from_address, to_address)
    );
    """)
    conn.commit()
    cur.close()
    conn.close()
    print("Tables created.")

if __name__ == "__main__":
    main()


✅ Tables created.


In [1]:

import json
import time
import psycopg2
import pandas as pd
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
import warnings


warnings.filterwarnings(
    "ignore",
    message=".*supports SQLAlchemy connectable.*"
)

# === 配置参数 ===
DB_NAME     = "fruit_chain"
DB_USER     = "postgres"
DB_PASSWORD = "lzq010921"
DB_HOST     = "localhost"
DB_PORT     = "5432"

RPC_URL        = "https://sepolia.infura.io/v3/7257d7dff2174065bbca319c11e7f4ae"
PERM_ADDR      = "0x275F2b6696bdF46bd03f81E9c7752b3c9B116097"#在这里修改permission address
PERM_ABI_PATH  = "./PermissionControl_ABI.json"
FRUIT_ABI_PATH = "./FruitTraceability_ABI.json"

def main():
    # 连接 DB
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD,
        host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()

    # 初始化 Web3 & 合约
    w3 = Web3(Web3.HTTPProvider(RPC_URL))
    w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

    perm_abi = json.load(open(PERM_ABI_PATH))
    perm = w3.eth.contract(address=w3.to_checksum_address(PERM_ADDR), abi=perm_abi)
    ft_addr = perm.functions.traceabilityContract().call()

    fruit_abi = json.load(open(FRUIT_ABI_PATH))
    fruit = w3.eth.contract(address=w3.to_checksum_address(ft_addr), abi=fruit_abi)

    # 从最近区块往前100块开始监听
    start_block = max(0, w3.eth.block_number - 100)
    filters = {
        "BatchRegistered":      fruit.events.BatchRegistered.create_filter(from_block=start_block),
        "StageRecorded":        fruit.events.StageRecorded.create_filter(from_block=start_block),
        "OwnershipTransferred": fruit.events.OwnershipTransferred.create_filter(from_block=start_block),
    }

    # 用于存储监听到的事件
    batches_records   = []
    stages_records    = []
    ownership_records = []

    print("Event listener started...")
    try:
        while True:
            for name, filt in filters.items():
                entries = filt.get_new_entries()
                for evt in entries:
                    args = evt["args"]
                    print(f"[{name}] -> {dict(args)}")

                    # 写库
                    if name == "BatchRegistered":
                        cur.execute(
                            "INSERT INTO batches(batch_id,metadata,current_owner,farmer) "
                            "VALUES (%s,%s,%s,%s) ON CONFLICT DO NOTHING",
                            (args["batchId"], args["metadata"], args["farmer"], args["farmer"])
                        )
                        # 累积到列表
                        batches_records.append({
                            "batch_id":      args["batchId"],
                            "metadata":      args["metadata"],
                            "current_owner": args["farmer"],
                            "farmer":        args["farmer"]
                        })

                    elif name == "StageRecorded":
                        cur.execute(
                            "INSERT INTO stages(batch_id,stage_type,location,timestamp,actor_address) "
                            "VALUES (%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING",
                            (args["batchId"], args["stage"], args["location"],
                             args["timestamp"], args["actor"])
                        )
                        stages_records.append({
                            "batch_id":      args["batchId"],
                            "stage_type":    args["stage"].name,  # 枚举名
                            "location":      args["location"],
                            "timestamp":     args["timestamp"],
                            "actor_address": args["actor"]
                        })

                    else:  # OwnershipTransferred
                        cur.execute(
                            "INSERT INTO ownership_transfers(batch_id,from_address,to_address) "
                            "VALUES (%s,%s,%s) ON CONFLICT DO NOTHING",
                            (args["batchId"], args["from"], args["to"])
                        )
                        ownership_records.append({
                            "batch_id":     args["batchId"],
                            "from_address": args["from"],
                            "to_address":   args["to"]
                        })

            conn.commit()
            time.sleep(2)

    except KeyboardInterrupt:
        # 用户中断后，直接用实际捕获到的事件生成 DataFrame 并打印
        print("\n Generating tables from real listened events...\n")

        print("=== Batches Table ===")
        print(pd.DataFrame(batches_records).to_markdown(index=False) or "(no batches)")

        print("\n=== Stages Table ===")
        print(pd.DataFrame(stages_records).to_markdown(index=False) or "(no stages)")

        print("\n=== Ownership Transfers ===")
        print(pd.DataFrame(ownership_records).to_markdown(index=False) or "(no transfers)")

    finally:
        cur.close()
        conn.close()
        print("\n Listener stopped, DB connection closed.")

if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named 'psycopg2'